In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [3]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#导入card_test数据(数据集1，导入1就不导入2)
card_test_name=['id','consume','where','how','time','amount','remainder']
card_test=pd.read_table(path + '/final_test/card_final_test.txt',sep=',',header=None,names=card_test_name)

#删除重复行并浏览前几行数据
card_test=card_test.drop_duplicates()
card_test.head()

,id,consume,where,how,time,amount,remainder
0,18021,POS消费,地点1,洗衣房,2013/09/01 00:00:16,1.20,284.68
2,9127,POS消费,地点2,淋浴,2013/09/01 00:00:48,0.90,117.05
4,10104,POS消费,地点3,淋浴,2013/09/01 00:01:59,0.50,403.61
6,16570,POS消费,地点4,淋浴,2013/09/01 00:02:06,0.70,120.27
8,18608,POS消费,地点5,开水,2013/09/01 00:02:47,0.01,4.41


In [2]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#导入card_test数据（数据集2）
card_test_name=['id','consume','where','how','time','amount','remainder']
card_test=pd.read_table(path + '/test/card_test.txt',sep=',',header=None,names=card_test_name)

#删除重复行并浏览前几行数据
card_test=card_test.drop_duplicates()
card_test.head()

,id,consume,where,how,time,amount,remainder
0,16894,POS消费,地点326,洗衣房,2013/09/01 00:00:18,0.10,432.76
2,10124,POS消费,地点102,淋浴,2013/09/01 00:00:18,0.60,113.26
4,5016,POS消费,地点36,开水,2013/09/01 00:00:19,0.01,134.37
6,776,POS消费,地点37,淋浴,2013/09/01 00:00:21,1.10,61.40
8,13112,POS消费,地点6,淋浴,2013/09/01 00:00:51,0.90,231.40


In [3]:
#表信息

card_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12029766 entries, 0 to 12392843
Data columns (total 7 columns):
id           int64
consume      object
where        object
how          object
time         object
amount       float64
remainder    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 734.2+ MB


In [4]:
#定义time转化函数

from datetime import datetime
def time_transform(x):
    return datetime.strptime(x,'%Y/%m/%d %H:%M:%S')

In [5]:
#转换字符串型的time成时间戳格式

card_test['time']=card_test['time'].apply(time_transform)

In [6]:
#时间格式转换

card_test['date']=card_test['time'].apply(lambda x:x.strftime('%m-%d-%Y'))
card_test['year']=card_test['time'].apply(lambda x:x.strftime('%Y'))
card_test['month']=card_test['time'].apply(lambda x:x.strftime('%m'))
card_test['hour']=card_test['time'].apply(lambda x:x.strftime('%H'))
card_test['weekday']=card_test['time'].apply(lambda x:x.strftime('%A'))

In [10]:
#上面数据处理花时很多，先保存处理后的结果

card_test.to_csv('input/card_test_time.csv',index=False)

In [7]:
#上面数据处理花时很多，先保存处理后的结果

card_test.to_csv('input/card_test_time_2.csv',index=False)

In [8]:
#hour

#将每天分成8个时段

a=['00','01']#熬夜
b=['06','07']#早餐
c=['08','09']#早餐
d=['11','12','13']#午饭
e=['17','18','19','20','21']#晚饭
f=['22','23']#宵夜
g=['02','03','04','05']#深夜
#h=['10','14','15','16']#其它
def hour_transform(hour):
    if hour in a:
        return 'a'
    elif hour in b:
        return 'b'
    elif hour in c:
        return 'c'
    elif hour in d:
        return 'd'
    elif hour in e:
        return 'e'
    elif hour in f:
        return 'f'
    elif hour in g:
        return 'g'
    else:
        return 'h'
    
card_test['hour']=card_test['hour'].apply(hour_transform)

#对hour进行one_hot编码得到hour新表
hour_dummies=pd.get_dummies(card_test['hour'])
hour_dummies.columns = ['hour_00_01','hour_06_07','hour_08_09','hour_11_13','hour_17_21','hour_22_23','hour_02_05','hour_other']

#连接两表
card_test = card_test.join(hour_dummies)

In [9]:
#weekday

#分为周末与非周末
a=['Monday','Tuesday','Wednesday','Thursday','Friday']
#b=['Saturday','Sunday']
def week_transform(weekday):
    if weekday in a:
        return 'wd'
    else:
        return 'vd'
    
card_test['weekday']=card_test['weekday'].apply(week_transform)

#对weekday进行one_hot编码得到weekday新表

weekday_dummies = pd.get_dummies(card_test['weekday'])
weekday_dummies.columns=['resting_day','working_day']
card_test=card_test.join(weekday_dummies)

In [10]:
#month

#分为在校，寒假与暑假

a=['01','02']#寒假
b=['07','08']#暑假
def month_transform(month):
    if month in a:
        return 'a'
    elif month in b:
        return 'b'
    else:
        return 'c'
    
card_test['month']=card_test['month'].apply(month_transform)

#对month进行one_hot编码得到where新表

month_dummies = pd.get_dummies(card_test['month'])
month_dummies.columns=['winter_vacation','summer_vacation','on_school']
card_test=card_test.join(month_dummies)

In [12]:
consume=card_test.groupby('consume')['id'].count()
consume

consume
POS充值           2
POS消费    11624621
交易冲正           25
卡充值        198982
卡冻结            56
卡挂失         11969
卡支取             2
卡片开户         9613
卡片销户         7764
卡补办          7770
卡解冻            56
卡解挂          4713
圈存补帐           10
圈存转账        74890
换卡           3741
支付领取        44769
更新卡信息       10998
车载消费           23
锁卡流水            1
Name: id, dtype: int64

In [13]:
#consume

#对consume进行one_hot编码得到consume新表
consume_dummies = pd.get_dummies(card_test['consume'])

#只保留使用数量多且关于消费的特称
consume_dummies.drop(['POS充值','交易冲正','卡冻结','卡挂失','卡支取','卡片开户','卡片销户','卡补办'
                      ,'卡解冻','卡解挂','圈存补帐','换卡','更新卡信息','车载消费','锁卡流水'], axis=1, inplace=True)
consume_dummies.columns = ['POS_recharge','card_recharge','load_tansfer','payment_collection']

#连接表
card_test = card_test.join(consume_dummies)

In [14]:
#how

#对consume进行one_hot编码得到consume新表

how_dummies = pd.get_dummies(card_test['how'])
how_dummies.columns=['other','library','water','office','printting','hospital','bus','landry','shower','market','canteen']
card_test = card_test.join(how_dummies)

In [15]:
#获得前十大最受欢迎的食堂以及所有食堂列表

card_test_canteen=card_test.loc[card_test['canteen']==1]
card_test_canteen=DataFrame(card_test_canteen.groupby('where')['where'].count())
card_test_canteen=card_test_canteen.sort(['where'],ascending=False)
card_test_canteen[:10]
#card_test_canteen.index

C:\Users\Wvv\Anaconda3\envs\py2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,where
where,
地点232,377715
地点118,146937
地点83,113861
地点61,89094
地点1155,81623
地点217,75674
地点72,74417
地点69,74111
地点60,70623


In [16]:
#获得前十大最受欢迎的非食堂地点及列表

card_test_where=card_test.loc[card_test['canteen']==0]
card_test_where=DataFrame(card_test_where.groupby('where')['where'].count())
card_test_where=card_test_where.sort(['where'],ascending=False)
card_test_where[:10]
#card_test_where.index

C:\Users\Wvv\Anaconda3\envs\py2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,where
where,
地点21,2190728
地点91,649240
地点188,183191
地点192,174941
地点26,139541
地点996,127612
地点6,113399
地点219,101279
地点248,86821


In [17]:
#where

a=['地点232','地点118','地点83','地点61','地点1155','地点217','地点69','地点72','地点60','地点841']
b=list(card_test_canteen.index)
c=['地点21','地点91','地点188','地点192','地点26','地点996','地点6','地点219','地点190','地点248']
#d=list(card_test_where.index)
def where_tansform(where):
    if where in a:
        return 'a'
    elif where not in a and where in b:
        return 'b'
    elif where in c:
        return 'c'
    else:
        return 'd'
    
card_test['where']=card_test['where'].apply(where_tansform)

#对where进行one_hot编码得到where新表

where_dummies = pd.get_dummies(card_test['where'])
where_dummies.columns = ['f_ten_canteen','other_canteen','f_ten_place','other_place']
card_test = card_test.join(where_dummies)

In [18]:
card_test.head()

,id,consume,where,how,time,amount,remainder,date,year,month,...,hospital,bus,landry,shower,market,canteen,f_ten_canteen,other_canteen,f_ten_place,other_place
0,16894,POS消费,d,洗衣房,2013-09-01 00:00:18,0.10,432.76,09-01-2013,2013,c,...,0,0,1,0,0,0,0,0,0,1
2,10124,POS消费,d,淋浴,2013-09-01 00:00:18,0.60,113.26,09-01-2013,2013,c,...,0,0,0,1,0,0,0,0,0,1
4,5016,POS消费,d,开水,2013-09-01 00:00:19,0.01,134.37,09-01-2013,2013,c,...,0,0,0,0,0,0,0,0,0,1
6,776,POS消费,d,淋浴,2013-09-01 00:00:21,1.10,61.40,09-01-2013,2013,c,...,0,0,0,1,0,0,0,0,0,1
8,13112,POS消费,c,淋浴,2013-09-01 00:00:51,0.90,231.40,09-01-2013,2013,c,...,0,0,0,1,0,0,0,0,1,0


In [54]:
#导出数据

#card_test.to_csv('input/card_test_f.csv',index=False)

In [19]:
card_test.to_csv('input/card_test_f_2.csv',index=False)